Create Material for layered QW Object

In [13]:
from solcore import si, material
from solcore.structure import Layer, Structure
import solcore.quantum_mechanics as QM
import matplotlib.pyplot as plt
from solcore.structure import TunnelJunction
from solcore.solar_cell_solver import default_options
from solcore.analytic_solar_cells import parametric_tunnel_junction
import solcore.poisson_drift_diffusion as PDD
import numpy as np


# First we create the materials we need
top = material("GaAs")(T=293, strained = False)
barrier = material("GaAsP")(T=293, P = 0.32, strained = False)
well = material("InGaAs")(T=293, In = 0.11, strained = False)
bottom = material("AlGaAs")(T=293, Al = 0.5, Strained = False) #doesn't recognize name pf compound

# As well as some of the layers
top_layer = Layer(width=si("10nm"), material=top)
barrier_layer = Layer(width=si("2.5nm"), material=barrier, role = 'barrier')
well_layer = Layer(width=si("8.5nm"), material=well, role = 'well')
bottom_barrier_layer = Layer(width=si("50nm"), material=bottom)
bottom_layer = Layer(width=si("50nm"), material=top)


# Layered quantum wells
layers = [top_layer, barrier_layer] + 10 * [well_layer, barrier_layer] + [bottom_barrier_layer, bottom_layer]

Create QW Layered Object

In [14]:
m_e = 9.1093837015*10**(-31) #kg
h_bar = 1.054571817E-34#in J*s
epsilon0 = 8.854187817 * 10**(-12) #C**2/(N*m**2)
epsilon = 11.48*epsilon0
effective_mass = 0.1*m_e
doping_concentration = 10**25 #m^-3
prefactor = (4/(3*h_bar))*((effective_mass*epsilon)/doping_concentration)**0.5

tunnel = TunnelJunction(v_peak=0.2, j_peak=100, v_valley=0.9, j_valley=10, prefactor=prefactor, j01=1e-21, kind='parametric')
parametric_tunnel_junction(tunnel, default_options)

v = tunnel.voltage

In [15]:
T = 300
interlayers = [barrier_layer, well_layer, barrier_layer]
wavelengths = np.linspace(350, 1050, 1001) * 1e-9

QW = PDD.QWunit(interlayers, T=T, repeat=10, substrate=top)

# We solve the quantum properties of the QW, leaving the default values of all parameters
QW_list = QW.GetEffectiveQW(wavelengths=wavelengths)

Solving QW properties...


In [18]:
from solcore.structure import Junction

T = 300

## Materials for the BOTTOM junction
window_bottom = material('GaInP')(T=T, Nd=5e24, In=0.49)
n_GaAs = material('GaAs')(T=T, Nd=1e24)
p_GaAs = material('GaAs')(T=T, Na=8e22)
bsf_bottom = material('GaInP')(T=T, Na=5e24, In=0.49)

GaAs_junction = Junction([Layer(width=60e-9, material=window_bottom, role="Window"),
                          Layer(width=1000e-9, material=n_GaAs, role="Emitter")] +
                         QW_list +
                         [Layer(width=2000e-9, material=p_GaAs, role="Base"),
                          Layer(width=200e-9, material=bsf_bottom, role="BSF")], sn=1e6, sp=1e6, T=T, kind='PDD')

## Materials for the TOP junction
window_top = material('AlInP')(T=T, Nd=5e23, Al=0.53)
n_GaInP = material('GaInP')(T=T, Nd=5e23, In=0.49)
p_GaInP = material('GaInP')(T=T, Na=8e22, In=0.49)
bsf_top = material('AlInP')(T=T, Na=5e24, Al=0.53)

GaInP_junction = Junction([Layer(width=30e-9, material=window_top, role="window"),
                         Layer(width=100e-9, material=n_GaInP, role="Emitter"),
                           Layer(width=2000e-9, material=p_GaInP, role="Base"),
                           Layer(width=100e-9, material=bsf_top, role="BSF")], sn=1e3, sp=1e3, T=T, kind='PDD')

Define Solar Cell

In [19]:
# antireflective coating
MgF2 = material('MgF2')()
ZnS = material('ZnScub')()

Arc1 = Layer(width=110e-9, material=MgF2, role="ARC1")
Arc2 = Layer(width=60e-9, material=ZnS, role="ARC2")

from solcore.solar_cell import SolarCell

my_solar_cell = SolarCell([Arc1,
                           Arc2,
                           GaInP_junction,
                           tunnel,
                           GaAs_junction],
                          T=T, substrate=n_GaAs)

In [20]:
from solcore.solar_cell_solver import solar_cell_solver
from solcore.light_source import LightSource

light_source = LightSource(source_type='standard', version='AM1.5g', x=wavelengths,
                           output_units='photon_flux_per_m', concentration=1)

solar_cell_solver(my_solar_cell, 'qe',
                  user_options={'light_source': light_source, 'wavelength': wavelengths, 'optics_method': 'TMM'})